# Project on testing ML techniques to identify YSOs in Spitzer IRAC data

## Breanna Crompvoets and Samuel Fielder

## Project Summary and Goals
Young Stellar Objects (YSOs) are newly forming stars which are yet to begin burning. They are split into different classes depending on their dust/gas envelope to protostar ratio; Class 0 having the greatest envelope and Class III no longer having an envelope. Due to the different ratios of envelope to protostar, each class appears differently in spectroscopy; thus the difference in fluxes between Spitzer IRAC bands are able to determine which class the data comes from. This project will focus on using the same data as Cornu and Montillaud (2021; CM21) to classify data points into three classes: Class I, Class II, and Contaminants. Only these three classes out of the original 9 available are chosen as Class 0 is too dusty to detect, and Class III is difficult to distinguish from regular stars. Furthermore, the contaminating classes (galaxies, shocks, stars, and PAHs) are of less concern -- we would like the algorithms to focus on distinguishing Class I and Class II from the rest. The original paper uses a multi-layer perceptron (MLP) with one hidden layer (20 neurons). Their results are presented in the below table.

|Class | Recall | Precision |
| --- | --- | --- |
|1 | 94.0% | 79.1 %|
|2 | 96.7% | 90.6% | 
|Other | 98.7%| 99.8%| 

The data for this project was pulled from https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/647/A116. These data include columns for four Spitzer IRAC bands (3.6 $\mu m$, 4.5 $\mu m$, 5.8 $\mu m$, and 8 $\mu m$) fluxes and errors, as well as from one Spitzer MIPS band (24 $\mu m$), along with the target values as determined via a manual classification scheme and the predicted data from CM21. We will only be using the four IRAC bands and their associated errors, as the MIPS band does not provide data for most objects. We use the same target values as they do for accurate comparison. 

This project seeks to use a multitude of algorithms learned over the semester to measure their effectiveness and compare it to the recreated MLP of CM21. These algorithms include: GridSearch with an SVC, GridSearch with a Logistic Regressor, a Stacking Ensemble with an SVC and a Logistic Regressor, a Gradient Boosting ensemble, a Random Forest ensemble, and an XGBoost ensemble. We also created our own MLP based off of their prescription. The workload was split as follows: B. Crompvoets completed the data cleaning and all other algorithms besides the MLP, and S. Fielder completed an MLP close to that of CM21, as well as creating a custom data loader/split. They communicated together on the best hyper-parameters to test.


## Import Libraries and set global variables

In [25]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

# classic ML libraries
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, recall_score, precision_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,  GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
import xgboost as xgb

# custom made libraries
from custom_dataloader import replicate_data

In [2]:
# settings for confusion matrix plots and classification reports
cm_blues = plt.cm.Blues
custom_labs = ['Class 1', 'Class 2', 'Others']

# Classical ML Techniques

For each of the below algorithms we ran a gridsearch over a wide variety of hyperparameters. These parameter dictionaries are commented out in each cell, and we use the best parameters as a sample here to show how each algorithm performed. This section of the project was conducted by B. Crompvoets.

We conducted these fits for each of three data splits:
* "75/25" -- here the data is split into 75% training, 25% test set.
* "300s" -- here the data is split such that 5 out of the 7 subclasses, each has 300 members in the training set (2 do not have enough members). The test set size is the same as CM21. The data is again split into only 3 classes to train/test.
* "CM21" -- here the data is split with the exact same values as CM21 provide in their paper.

An example of a run with the 75/25 split is given below, with best results for each of the other runs commented out, as well as the parameter grid tested for each GridSearch. The results of all runs will be included at the end of this document.

## Loading Data Set

In [93]:
# data load
X = np.load("../Data_and_Results/Input_Class_AllClasses_Sep.npy")
# Y = np.load("../Data_and_Results/Target_Class_AllClasses_Sep.npy") # For original targets via Gutermuth 2009 Method
Y = np.load("../Data_and_Results/Pred_Class_AllClasses_Sep.npy") # For predicted targets from CM21

# the amounts below are how many of each class of object you want in the training set and validation set - leftover amounts given to testing set

# # CM21 Split
# amounts_train = [331,1141,231,529,27,70,1257]
# amounts_val = [82, 531, 104, 278, 6, 17, 4359]
# amounts_train = [331,1141,231+529+27+70+1257]
# amounts_val = [82, 531, 104+278+6+17+4359]


# 300s Split
# amounts_train = [300,300,300,300,27,70,300]
# amounts_val = [82, 531, 104, 278, 6, 17, 4359]
# amounts_train = [300,300,300+300+27+70+300]
# amounts_val = [82, 531, 104+278+6+17+4359]


# 75/25 Split
# amounts_train = [311,1994,391,1043,25,66,21796] #75/25 train
# amounts_val = [103,665,130,348,9,22,5449] #75/25 val
amounts_train = [311,1994,391+1043+25+66+21796] #75/25 train
amounts_val = [103,665,130+348+9+22+5449] #75/25 val

In [78]:
# Get values of true and predicted values for XGB to understand what objects are a source of contamination

# inputs = np.concatenate((inp_tr,inp_va,inp_te))
# targets = np.concatenate((tar_tr,tar_va,tar_te))
# np.save("XGB_Val_G-targets7.npy",targets)
# tar_tr = np.where(tar_tr<2,tar_tr,2)
# tar_va = np.where(tar_va<2,tar_va,2)
# tar_te = np.where(tar_te<2,tar_te,2)
# targets = np.concatenate((tar_tr,tar_va,tar_te))
# np.save("XGB_Val_G-targets2.npy",targets)

# xgbcl = xgb.XGBClassifier(max_depth=7,sampling_method='uniform',subsample=0.5,use_label_encoder=False,eval_metric='mlogloss')
# xgbcl.fit(inp_tr,tar_tr)
# pred_va = xgbcl.predict(inputs)
# np.save("XGB_300s_G-targets_VPred.npy",pred_va)

In [45]:
def bootstrap_estimate(estimator, X, Y, amounts_train, amounts_val, n_splits=100):
                          
    scoresA = []
    scoresP = []
    scoresR = []
    
    for n in range(0,n_splits):
        inp_tr, tar_tr, inp_va, tar_va, inp_te, tar_te = replicate_data(X, Y, 'three', amounts_train, amounts_val, random.randint(0,1000))
        scaler_S = StandardScaler().fit(inp_tr)
        inp_tr = scaler_S.transform(inp_tr)
        inp_va = scaler_S.transform(inp_va)
        # inp_te = scaler_S.transform(inp_te)
        estimator.fit(inp_tr, tar_tr.ravel())  
        pred_va = estimator.predict(inp_va)
        scoresA.append(accuracy_score(tar_va,pred_va))
        scoresR.append(recall_score(tar_va,pred_va,average=None,zero_division=1))  
        scoresP.append(precision_score(tar_va,pred_va,average=None,zero_division=1)) 
        
    scoresR = list(map(list, zip(*scoresR)))
    scoresP = list(map(list, zip(*scoresP)))

    estimateA = np.mean(scoresA)*100.
    stderrA = np.std(scoresA)*100.
    
    estimateR = [np.mean(scoresR[0])*100.,np.mean(scoresR[1])*100.,np.mean(scoresR[2])*100.]
    stderrR = [np.std(scoresR[0])*100.,np.std(scoresR[1])*100.,np.std(scoresR[2])*100.]
    
    estimateP = [np.mean(scoresP[0])*100.,np.mean(scoresP[1])*100.,np.mean(scoresP[2])*100.]
    stderrP = [np.std(scoresP[0])*100.,np.std(scoresP[1])*100.,np.std(scoresP[2])*100.]
    
    return estimateR, stderrR, estimateP, stderrP, estimateA, stderrA

def bootstrap_to_file(file, estimator, X, Y, amounts_train, amounts_val, n_splits=200):
    estR, stderrR, estP, stderrP, estA, stderrA = bootstrap_estimate(estimator, X, Y, amounts_train, amounts_val, n_splits=n_splits)
    classes = ["Class I", "Class II", "Contaminants"]
    for i, cl in enumerate(classes):
        if i==1:
            file.write(cl+"& $"+"{:.1f}".format(estR[i])+"\pm"+"{:.1f}".format(stderrR[i])+"$ & $"+
                "{:.1f}".format(estP[i])+"\pm"+"{:.1f}".format(stderrP[i])+"$ & $"+"{:.1f}".format(estA)+"\pm"+"{:.1f}".format(stderrA)+"$ // \n")
        else:
            file.write(cl+"& $"+"{:.1f}".format(estR[i])+"\pm"+"{:.1f}".format(stderrR[i])+"$ & $"+
                "{:.1f}".format(estP[i])+"\pm"+"{:.1f}".format(stderrP[i])+"$&// \n")


In [94]:
f = open("../PRA_Scores/PRAScores_7525_C-targets.txt","w")
f.write("This is the correct bootstrapped values now. \n")

# f.close()

46

## Logistic Regression


Specifying logistic regression
logreg = LogisticRegression()

hyperparameters tested over initially
param_grid = [{'penalty': ['l1'], 'max_iter': np.arange(300,1500,100),
        'solver': ['liblinear', 'saga'], 'tol': np.arange(0.0001,0.01,0.0005)},
        {'penalty': ['l2'], 'max_iter': np.arange(300,1500,100),
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'tol': np.arange(0.0001,0.01,0.0005)},
        {'penalty': ['elasticnet'], 'max_iter': np.arange(100,2000,100), 'l1_ratio': np.arange(0.1,1.,0.1),
        'solver': ['saga'], 'tol': np.arange(0.0001,0.01,0.0005)}]

grid = GridSearchCV(logreg,param_grid=param_grid, verbose=1)

Run the data through the grid to find optimal results
grid.fit(inp_tr, tar_tr.ravel())

In [95]:
%%time
# 75/25
logreg = LogisticRegression('l1',max_iter=700,solver='saga',tol=0.0001)
# 300s
# logreg = LogisticRegression('l1',max_iter=300,solver='saga',tol=0.0016)
# CM21
# logreg = LogisticRegression('l1',max_iter=600,solver='saga',tol=0.0001)

CPU times: user 12 µs, sys: 1e+03 ns, total: 13 µs
Wall time: 14.1 µs


In [96]:
f.write("LR Recall & Precision & Accuracy\n")
bootstrap_to_file(f, logreg, X, Y, amounts_train, amounts_val, n_splits=100)

/Users/breannacrompvoets/miniforge3/envs/SF/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/miniforge3/envs/SF/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/miniforge3/envs/SF/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/miniforge3/envs/SF/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/breannacrompvoets/miniforge3/envs/SF/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the 

## SVM

Hyperparameters tested over initially
param_grid = [{'kernel':['rbf','sigmoid','linear','poly'], 'gamma':['auto','scale'], 'C':np.arange(0.1,1.,0.1)}]

In [97]:
%%time
# Final hyperparameters
# 75/25
svc = SVC(kernel='rbf',gamma='auto',C=0.9)
# 300s
# svc = SVC(kernel='linear',gamma='auto',C=0.8)
# CM21
# svc = SVC(kernel='rbf',gamma='auto',C=0.9)

CPU times: user 17 µs, sys: 1 µs, total: 18 µs
Wall time: 17.9 µs


In [98]:
f.write("SVC Recall & Precision & Accuracy\n")
bootstrap_to_file(f, svc, X, Y, amounts_train, amounts_val, n_splits=100)

## SVM/LR Stacking Ensemble
No GridSearch, uses best pars as defined previously. Adding multiple SVC's does not improve results. 

In [99]:
%%time

# Specify Gradient Boost
# 75/25
estimators = [('svc', SVC(kernel='rbf',gamma='auto',C=0.9,random_state=42))]
# 300s
# estimators = [('svc', SVC(C=0.8, gamma='auto', kernel='linear',random_state=42))]
# CM21 
# estimators = [('svc', SVC(C=0.9, gamma='auto', kernel='rbf',random_state=42))]

# As the parameters for the Logistic Regression didn't change much, we use the best pars from the first trial.
stacl = StackingClassifier(estimators=estimators,
                           final_estimator=LogisticRegression(penalty = 'l1', max_iter = 500, solver ='saga', tol =0.0001))

CPU times: user 27 µs, sys: 7 µs, total: 34 µs
Wall time: 35 µs


In [100]:
f.write("Stack Recall & Precision & Accuracy\n")
bootstrap_to_file(f, stacl, X, Y, amounts_train, amounts_val, n_splits=100)

## Gradient Boosting

Hyperparameters tested over initially
param_grid = [{'n_estimators': np.arange(50,250,50),'subsample':[0.5,1.0],
              'criterion':['friedman_mse'],'n_iter_no_change':[5],'warm_start':[True,False],
              'max_depth':np.arange(1,11,2),'max_features': ['sqrt','log2']}]

In [101]:
%%time
# Final hyperparameters
# 75/25
boostcl = GradientBoostingClassifier(criterion='friedman_mse',max_depth=9,max_features='log2',
                n_estimators=50,n_iter_no_change=5,subsample=1.0,warm_start=True)

# 300s
# boostcl = GradientBoostingClassifier(criterion='friedman_mse',max_depth=5,max_features='log2',
#                 n_estimators=150,n_iter_no_change=5,subsample=1.0,warm_start=False)

# CM21
# boostcl = GradientBoostingClassifier(criterion='friedman_mse',max_depth=7,max_features='log2',
#                 n_estimators=200,n_iter_no_change=5,subsample=1.0,warm_start=True)

CPU times: user 180 µs, sys: 11 µs, total: 191 µs
Wall time: 193 µs


In [102]:
f.write("GB Recall & Precision & Accuracy\n")
bootstrap_to_file(f, boostcl, X, Y, amounts_train, amounts_val, n_splits=100)

### XGBoost
Hyperparameters tested over initially
param_grid = [{'subsample':[0.5,1.0],'max_depth':np.arange(1,11,2),'sampling_method':['uniform']}]


In [103]:
# 75/25
xgbcl = xgb.XGBClassifier(max_depth=9,sampling_method='uniform',subsample=0.5,use_label_encoder=False,eval_metric='mlogloss')
# 300s
# xgbcl = xgb.XGBClassifier(max_depth=7,sampling_method='uniform',subsample=0.5,use_label_encoder=False,eval_metric='mlogloss')
# CM21
# xgbcl = xgb.XGBClassifier(max_depth=9,sampling_method='uniform',subsample=1.0,use_label_encoder=False,eval_metric='mlogloss')

In [104]:
f.write("XGB Recall & Precision & Accuracy\n")
bootstrap_to_file(f, xgbcl, X, Y, amounts_train, amounts_val, n_splits=100)

## Random Forest

Hyperparameters tested over initially
param_grid = [{'class_weight': ['balanced_subsample','balanced'], 'n_estimators': np.arange(50,250,50),
        'criterion': ['gini', 'entropy'], 'max_features': ['sqrt','log2'], 'oob_score':[True,False]}]

In [105]:
# Final hyperparameters
# 75/25
rfcl = RandomForestClassifier(class_weight='balanced_subsample',criterion='entropy',max_features='log2',n_estimators=150,oob_score=False)
# 300s
# rfcl = RandomForestClassifier(class_weight='balanced',criterion='entropy',max_features='log2',n_estimators=50,oob_score=False)
# CM21
# rfcl = RandomForestClassifier(class_weight='balanced',criterion='entropy',max_features='log2',n_estimators=100,oob_score=True)

In [106]:
f.write("RF Recall & Precision & Accuracy\n")
bootstrap_to_file(f, rfcl, X, Y, amounts_train, amounts_val, n_splits=100)

In [107]:
f.close()